In [1]:
pip install sdv torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [43]:
import pandas as pd
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

In [16]:
transactions_df = pd.read_csv('/content/transactions.csv')
training_df = pd.read_csv('/content/training_data.csv')

In [23]:
transactions_df['user_id'] = transactions_df['subject'].str.replace('User ', '').astype(int)

In [24]:
transactions_df.head()

,subject,verb,object,time,amount,mask,user_id
0,User 40578,bought,book,2023-01-11,1524,"[1, 1, 1, 1, 1]",40578
1,User 94768,paid,laptop,2022-11-23,-1,"[1, 1, 1, 1, 0]",94768
2,User 65142,paid,service,2023-03-25,-1,"[1, 1, 1, 1, 0]",65142
3,User 10499,returned,book,2023-12-19,-1,"[1, 1, 1, 1, 0]",10499
4,User 89538,ordered,phone,2025-07-13,3949,"[1, 1, 1, 1, 1]",89538


In [18]:
training_df.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
training_df.head()

,user_id,verb_bought,verb_cancelled,verb_ordered,verb_paid,verb_returned,verb_sold,object_accessory,object_book,object_laptop,object_phone,object_product,object_service,mask_time,mask_amount,unix_time_norm,amount_norm
0,40578,True,False,False,False,False,False,False,True,False,False,False,False,1,1,0.991470,0.652465
1,94768,False,False,False,True,False,False,False,False,True,False,False,False,1,0,0.991085,0.000000
2,65142,False,False,False,True,False,False,False,False,False,False,False,True,1,0,0.992044,0.000000
3,10499,False,False,False,False,True,False,False,True,False,False,False,False,1,0,0.994159,0.000000
4,89538,False,False,True,False,False,False,False,False,False,True,False,False,1,1,0.998656,0.894989


In [20]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           20000 non-null  int64  
 1   verb_bought       20000 non-null  bool   
 2   verb_cancelled    20000 non-null  bool   
 3   verb_ordered      20000 non-null  bool   
 4   verb_paid         20000 non-null  bool   
 5   verb_returned     20000 non-null  bool   
 6   verb_sold         20000 non-null  bool   
 7   object_accessory  20000 non-null  bool   
 8   object_book       20000 non-null  bool   
 9   object_laptop     20000 non-null  bool   
 10  object_phone      20000 non-null  bool   
 11  object_product    20000 non-null  bool   
 12  object_service    20000 non-null  bool   
 13  mask_time         20000 non-null  int64  
 14  mask_amount       20000 non-null  int64  
 15  unix_time_norm    20000 non-null  float64
 16  amount_norm       20000 non-null  float6

In [28]:
merged_df = pd.merge(
    training_df,
    transactions_df,
    on="user_id",
    how="left"
).drop(columns=["subject"])

In [32]:
merged_df.head()

,user_id,verb_bought,verb_cancelled,verb_ordered,verb_paid,verb_returned,verb_sold,object_accessory,object_book,object_laptop,...,object_service,mask_time,mask_amount,unix_time_norm,amount_norm,verb,object,time,amount,mask
0,40578,True,False,False,False,False,False,False,True,False,...,False,1,1,0.991470,0.652465,bought,book,2023-01-11,1524,"[1, 1, 1, 1, 1]"
1,94768,False,False,False,True,False,False,False,False,True,...,False,1,0,0.991085,0.000000,paid,laptop,2022-11-23,-1,"[1, 1, 1, 1, 0]"
2,65142,False,False,False,True,False,False,False,False,False,...,True,1,0,0.992044,0.000000,paid,service,2023-03-25,-1,"[1, 1, 1, 1, 0]"
3,10499,False,False,False,False,True,False,False,True,False,...,False,1,0,0.994159,0.000000,returned,book,2023-12-19,-1,"[1, 1, 1, 1, 0]"
4,10499,False,False,False,False,True,False,False,True,False,...,False,1,0,0.994159,0.000000,cancelled,book,2023-04-22,-1,"[1, 1, 1, 1, 0]"


In [33]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(merged_df)

In [40]:
metadata.update_column(
    column_name='user_id',
    sdtype='id',
    regex_format=r'User \d{5}'
)

metadata.update_column(
    column_name='verb_bought',
    sdtype='boolean'
)

metadata.update_column(
    column_name='amount_norm',
    sdtype='numerical'
)

In [46]:
merged_df['synthetic_id'] = range(1, len(merged_df) + 1)
metadata.add_column('synthetic_id', sdtype='id')
metadata.set_primary_key('synthetic_id')

/usr/local/lib/python3.11/dist-packages/sdv/metadata/single_table.py:778: UserWarning: There is an existing primary key 'user_id'. This key will be removed.
  warnings.warn(


In [47]:
metadata.validate()

In [48]:
synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=False,
    epochs=100,
    batch_size=1000,
    verbose=True
)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [49]:
synthesizer.fit(merged_df)

PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name   Est # of Columns (CTGAN)
verb_bought            2
verb_cancelled         2
verb_ordered           2
verb_paid              2
verb_returned          2
verb_sold              2
object_accessory       2
object_book            2
object_laptop          2
object_phone           2
object_product         2
object_service         2
mask_time              2
mask_amount            2
unix_time_norm         11
amount_norm            11
verb                   6
object                 6
time                   1462
amount                 11
mask                   4

We recommend preprocessing discrete columns that can have many values, using 'update_transformers'. Or you may drop columns that are not necessary to model. (Exit this script using ctrl-C)


Gen. (-0.87) | Discrim. (-0.31): 100%|██████████| 100/100 [1:45:39<00:00, 63.39s/it]


In [50]:
synthetic_data = synthesizer.sample(num_rows=1000)

In [54]:
synthetic_data.head()

,user_id,verb_bought,verb_cancelled,verb_ordered,verb_paid,verb_returned,verb_sold,object_accessory,object_book,object_laptop,...,mask_time,mask_amount,unix_time_norm,amount_norm,verb,object,time,amount,mask,synthetic_id
0,User 00636,False,False,True,False,True,False,True,False,False,...,0,1,0.021291,0.979915,bought,accessory,[PAD],-4242,"[1, 1, 1, 0, 1]",14702678
1,User 00163,False,False,False,False,False,False,True,False,False,...,1,1,0.985889,0.540931,bought,phone,2022-06-30,351,"[1, 1, 1, 1, 1]",1688051
2,User 00890,False,False,False,False,False,True,False,False,False,...,1,0,0.006445,0.002089,cancelled,service,2022-11-18,-31,"[1, 1, 1, 1, 0]",5506731
3,User 00348,False,False,False,False,True,False,False,False,False,...,1,1,0.997406,0.159413,bought,book,2022-01-14,2538,"[1, 1, 1, 1, 0]",972145
4,User 00606,False,False,False,False,True,False,True,False,True,...,0,1,0.017769,0.781084,cancelled,book,[PAD],3480,"[1, 1, 1, 0, 1]",2185867
